# AWS Basic setup

Enable access to Amazon services like S3:
- get_execution_role: allow Amazon SageMaker to assume the role created during instance creation and accesses resources on your behalf.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import boto3
from sagemaker import get_execution_role

role = get_execution_role()

#### Define a bucket
This will hosts the dataset that will be used.

In [2]:
bucket = 'dataen-sagemaker-dev/datasets/cal-tech' 

#### Define the containers
Containers (docker containers) as the training job defined in this notebook will run in the container for your region.

In [3]:
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/image-classification:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/image-classification:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/image-classification:latest'}
training_image = containers[boto3.Session().region_name]

#### Import dataset

In [4]:
import os
import re
import urllib.request
import boto3


def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)


def upload_to_s3(bucket, file, channel):
    """
    Stores the file in the 'channel' folder within the specified bucket
    If bucket contains folders in the path that is 
    """
    subfolder = None
    if "/" in bucket:
        buc_folders = bucket.split('/')
        bucket = buc_folders[0]
        subfolder = buc_folders[1:]
    if isinstance(bucket, list):
        bucket, subfolder = bucket[0], bucket[1:]
        
    if subfolder:
        key = "/".join(subfolder) + "/" + channel + "/" + file
    else:
        key = channel + "/" + file
    
    # Read file
    with open(file, "rb") as data:
        s3 = boto3.resource('s3')
        s3.Bucket(bucket).put_object(Key=key, Body=data)

In [5]:
# caltech-256

# download('http://data.mxnet.io/data/caltech-256/caltech-256-60-train.rec')
# upload_to_s3('train', 'caltech-256-60-train.rec', bucket)
# download('http://data.mxnet.io/data/caltech-256/caltech-256-60-val.rec')
# upload_to_s3('validation', 'caltech-256-60-val.rec', bucket)

### Train a model in SageMaker

We need to create a training job. A training job includes the following information:

- S3 Bucket (input data) with the training data.
- S3 Bucket (output) for the model, data, etc.
- Compute resources to use in SageMaker (ML compute instances managed by Amazon SageMaker).
- ECR (Elastic Container Registry) path where the training code is stored.

For example we can pass the image classifier (ResNet) built in Amazon SageMaker. The checkpoint_frequency determines the frequency by which model files are stored during training. If we only need the final model file, we set it equal to the number of epochs.

Please make a note of **job_name_prefix, S3OutputPath, InstanceType, InstanceCount**.

In [6]:
# Model and Training Parameters

buc_folders = bucket.split('/')
if len(buc_folders) > 1:
    bucket, subfolders = buc_folders[0], "/".join(buc_folders[1:])
else:
    subfolders = "datasets/cal-tech"

# The algorithm supports multiple network depth (number of layers). They are 18, 34, 50, 101, 152 and 200
num_layers = "18" 
# we need to specify the input image shape for the training data
image_shape = "3,224,224"
# we also need to specify the number of training samples in the training set
# for caltech it is 15420
num_training_samples = "15420"
# specify the number of output classes
num_classes = "257"
# batch size for training
mini_batch_size =  "128"
# number of epochs
epochs = "3"
# learning rate
learning_rate = "0.1"
#optimizer
optimizer ='Adam'
#checkpoint_frequency
checkpoint_frequency = epochs     # We just want the final model so equal to epochs
#scheduler_step
lr_scheduler_step="30,90,180"
#scheduler_factor
lr_scheduler_factor="0.1"
#augmentation_type
augmentation_type="crop_color_transform"

### Pack parameters

Get all parameters into a dictionary structure: Job name, buckets, instance...


In [7]:
import time
import boto3
from time import gmtime, strftime

s3 = boto3.client('s3')
# create unique job name 
job_name_prefix = 'ImageClassificationResNet'
timestamp = time.strftime('-%Y-%m-%dT%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp

training_params = \
{
    # specify the training docker image
    "AlgorithmSpecification": {
        "TrainingImage": training_image,
        "TrainingInputMode": "File"
    },
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": f"s3://{bucket}/{subfolders}"
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.p2.8xlarge",
        "VolumeSizeInGB": 50
    },
    "TrainingJobName": job_name,
    "HyperParameters": {
        "image_shape": image_shape,
        "num_layers": str(num_layers),
        "num_training_samples": str(num_training_samples),
        "num_classes": str(num_classes),
        "mini_batch_size": str(mini_batch_size),
        "epochs": str(epochs),
        "learning_rate": str(learning_rate),
        "lr_scheduler_step": str(lr_scheduler_step),
        "lr_scheduler_factor": str(lr_scheduler_factor),
        "augmentation_type": str(augmentation_type),
        "checkpoint_frequency": str(checkpoint_frequency),
        "augmentation_type" : str(augmentation_type)
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 360000
    },
#Training data should be inside a subdirectory called "train"
#Validation data should be inside a subdirectory called "validation"
#The algorithm currently only supports fullyreplicated model (where data is copied onto each machine)
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": f's3://{bucket}/train/',
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "application/x-recordio",
            "CompressionType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": f's3://{bucket}/validation/',
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "application/x-recordio",
            "CompressionType": "None"
        }
    ]
}
print(f'Training job name: {job_name}')
print(f"\nInput Data Location: \n  {training_params['InputDataConfig'][0]['DataSource']['S3DataSource']}")
print(f"\nOutput Location: {training_params['OutputDataConfig']['S3OutputPath']}")

Training job name: ImageClassificationResNet-2019-10-31T08-45-28

Input Data Location: 
  {'S3DataType': 'S3Prefix', 'S3Uri': 's3://dataen-sagemaker-dev/train/', 'S3DataDistributionType': 'FullyReplicated'}

Output Location: s3://dataen-sagemaker-dev/datasets/cal-tech


### Launch the job and check status

In [15]:
# create the Amazon SageMaker training job
sagemaker = boto3.client(service_name='sagemaker')
sagemaker.create_training_job(**training_params)

# confirm that the training job has started
status = sagemaker.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print(f'Training job current status: {status}')

try:
    # wait for the job to finish and report the ending status
    sagemaker.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=job_name)
    training_info = sagemaker.describe_training_job(TrainingJobName=job_name)
    status = training_info['TrainingJobStatus']
    print("Training job ended with status: " + status)
except:
    print('Training failed to start')
     # if exception is raised, that means it has failed
    message = sagemaker.describe_training_job(TrainingJobName=job_name)['FailureReason']
    print(f'Training failed with the following error: {message}')

Training job current status: InProgress
Training job ended with status: Completed


### Create Lambda service for the model

In [9]:
import os
import greengrasssdk
from threading import Timer
import time
import awscam
import cv2
import mo
from threading import Thread

# Creating a greengrass core sdk client
client = greengrasssdk.client('iot-data')

More at ["build your own object classification model"](https://aws.amazon.com/blogs/machine-learning/build-your-own-object-classification-model-in-sagemaker-and-import-it-to-deeplens/)